# AI Census

AI Project, which predicts whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset.

## Data Exploration

The data exploration phase involves analyzing the dataset to understand the distribution of the variables, identifying any missing or outliers, and determining which features are relevant for the prediction phase.

## Data Visualization

Data visualization helps to understand the relationship between different variables and identify patterns in the data.
It allows to detect outliers and trends that might not be apparent from a simple statistical summary.

## Data Preprocessing

Data preprocessing is the process of cleaning and formatting the data to make it suitable for training the model.
This might include filling in missing values, normalizing the data, and encoding categorical variables.

## Training

The training phase is where the model is trained to fit to the data provided by the dataset, using a set of parameters called hyperparameters.
The goal is to find the optimal set of hyperparameters that will produce the best model for the prediction task.
But you also have to be careful about an overfitting or underfitting model.

## Hyperparameter Tuning

Hyperparameter tuning is the process of fine-tuning the model's hyperparameters to improve its performance on the prediction task.

## Evaluation

The evaluation phase involves assessing the performance of the model on a set of unknown realistic data to the model.
This process is usually done by comparing the predicted values to the true values, and calculating metrics such as accuracy, precision, and recall.